In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb
import lightgbm as lgb
import tensorflow as tf
import tensorflow.keras as keras 

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier

from keras.constraints import maxnorm

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import fbeta_score, make_scorer
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV


%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import imblearn

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv("music_scaled_train_pca_75.csv", header=0)
test_df = pd.read_csv("music_scaled_test_pca_25.csv", header=0)

In [3]:
X_train_df = train_df.drop(columns=['genre'])
X_test_df = test_df.drop(columns=['genre'])
y_train_df = train_df['genre']
y_test_df = test_df['genre']

In [4]:
label_num = train_df.groupby('genre')['genre'].count().count()
label_num

121

In [5]:
test_df.groupby('genre')['genre'].count().count()

121

In [12]:
#functions

def loss_scorer(y_true, y_pred, eps=1e-15, normalize=True, sample_weight=None, labels=None):
    return log_loss(y_true, y_pred, eps, normalize, sample_weight, labels=__LABELS__)

def search_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=10, scoring_fit='neg_mean_squared_error',
                       do_probabilities = False, search_mode = 'GridSearchCV', n_iterations = 0, n_jobs=-1):
    fitted_model = None
    
    if(search_mode == 'GridSearchCV'):
        gs = GridSearchCV(
            estimator=model,
            param_grid=param_grid, 
            cv=cv, 
            n_jobs=n_jobs, 
            scoring=scoring_fit,
            verbose=2
        )
        fitted_model = gs.fit(X_train_data, y_train_data)

    elif (search_mode == 'RandomizedSearchCV'):
        rs = RandomizedSearchCV(
            estimator=model,
            param_distributions=param_grid, 
            cv=cv,
            n_iter=n_iterations,
            n_jobs=n_jobs, 
            scoring=scoring_fit,
            verbose=2
        )
        fitted_model = rs.fit(X_train_data, y_train_data)
    
    
    if(fitted_model != None):
        if do_probabilities:
            pred = fitted_model.predict_proba(X_test_data)
        else:
            pred = fitted_model.predict(X_test_data)
            
        return fitted_model, pred

In [7]:
# Deep Learning model base
# Function to create model, required for KerasClassifier
def create_model(
    activation1='relu',
    activation2='relu',
    activation3='relu',
    activation4='relu',
    activation5='relu',
    dropout_rate=0.0,
    init_mode='uniform',
    weight_constraint=0,
    optimizer='adam',
    lr=0.01,
    momentum=0):
    # create model
    model = Sequential()
    model.add(Dense(1028, input_dim=input_dim, 
                    kernel_initializer=init_mode,
                    activation=activation1, 
                    kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dense(512, activation=activation2))
    model.add(Dense(256, activation=activation3))
    model.add(Dense(128, activation=activation4))
    model.add(Dropout(dropout_rate))
    model.add(Dense(label_num, 
                    kernel_initializer=init_mode, 
                    activation=activation5))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', 
                  optimizer=optimizer, 
                  metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7


#gridsearch area
activation1 = ['relu', 'tanh', 'sigmoid'
              , 'hard_sigmoid', 'linear'
              , 'softmax']
activation2 = ['relu', 'tanh', 'sigmoid'
              , 'hard_sigmoid', 'linear'
              , 'softmax']
activation3 = ['relu', 'tanh', 'sigmoid'
              , 'hard_sigmoid', 'linear'
              , 'softmax']
activation4 = ['relu', 'tanh', 'sigmoid'
              , 'hard_sigmoid', 'linear'
              , 'softmax']
activation5 = ['relu', 'tanh', 'sigmoid'
              , 'hard_sigmoid', 'linear'
              , 'softmax']
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
dropout_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
weight_constraint = [1, 2, 3, 4, 5]
neurons = [1, 5, 10, 15, 20, 25, 30]
init_mode = ['uniform', 'lecun_uniform', 'normal'
             , 'zero', 'glorot_normal', 'glorot_uniform'
             , 'he_normal', 'he_uniform']

optimizer = ['SGD', 'RMSprop', 'Adagrad'
             , 'Adadelta', 'Adam', 'Adamax'
             , 'Nadam']

epochs = [1, 10, 50, 100]
batch_size = [128, 512, 1024, 2048, 4096]
#param_grid = dict(epochs=epochs, batch_size=batch_size)

#reduce or expand this based on need and runtime
param_grid = dict(epochs=epochs
                  , batch_size=batch_size
                  , activation1=activation1
                  , activation2=activation2
                  , activation3=activation3
                  , activation4=activation4
                  , activation5=activation5
                  , dropout_rate=dropout_rate
                  , weight_constraint=weight_constraint
                  , init_mode=init_mode
                  , optimizer=optimizer
                 )

In [8]:
#do some oversampling to balance classes
from collections import Counter
from imblearn.over_sampling import SMOTE

sm = SMOTE()
# fit and apply the transform
X_over, y_over = sm.fit_resample(X_train_df, y_train_df)

#this might be dataleakage. We may need to oversample after split

In [9]:
print(Counter(y_over))

Counter({'Dance': 8049, 'Blues': 8049, 'Lo-Fi': 8049, 'Noise': 8049, 'Pop': 8049, 'Rock': 8049, 'Avant-Garde': 8049, 'Garage': 8049, 'Electronic': 8049, 'Unclassifiable': 8049, 'Novelty': 8049, 'Chiptune': 8049, 'Singer-Songwriter': 8049, 'Classical': 8049, 'Trip-Hop': 8049, 'Field Recordings': 8049, 'Punk': 8049, 'Soundtrack': 8049, 'Industrial': 8049, 'Reggae - Dub': 8049, 'Hip-Hop': 8049, 'Folk': 8049, 'Choral Music': 8049, 'Breakbeat': 8049, 'Jazz': 8049, 'Psych-Folk': 8049, 'Post-Punk': 8049, 'Spoken Weird': 8049, 'Post-Rock': 8049, 'Ambient Electronic': 8049, 'Experimental': 8049, 'Indie-Rock': 8049, 'Drone': 8049, 'International': 8049, 'Europe': 8049, 'Experimental Pop': 8049, 'Sound Collage': 8049, 'Grindcore': 8049, 'Sound Art': 8049, 'Audio Collage': 8049, 'Soul-RnB': 8049, 'Psych-Rock': 8049, 'OTHER': 8049, 'Hardcore': 8049, 'Lounge': 8049, 'Downtempo': 8049, 'Ambient': 8049, 'Easy Listening': 8049, 'Country': 8049, 'Loud-Rock': 8049, 'Metal': 8049, 'Electroacoustic': 8049,

In [10]:
#swap to oversampled dataframes
#comment out if you dont want oversampling
X_train_df = X_over
y_train_df = y_over

#Encode labels since tensorflow expects integers
encoder = LabelEncoder()
y = encoder.fit_transform(y_train_df)

#create global variable for this
#this isnt good practice but whatever
global __LABELS__
__LABELS__ = list(set(y))

In [13]:
# spliting of dataset into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X_train_df
                                                    , y
                                                    , test_size=0.2
                                                    , stratify=y)

#use RepeatedStratifiedKFold to roughly ensure that each 
#fold is representative of all strata of the data
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3)

# create Keras model
input_dim = X_train.shape[1]
model = KerasClassifier(build_fn=create_model)

model, pred = search_pipeline(X_train
                              , X_test
                              , y_train
                              , y_test
                              ,model
                              ,param_grid
                              ,cv=cv
                              ,scoring_fit=make_scorer(loss_scorer, 
                                                      greater_is_better=False, 
                                                      needs_proba=True)
                              #,search_mode = 'GridSearchCV'
                              ,search_mode = 'RandomizedSearchCV'
                              ,n_iterations = 100
                              ,n_jobs = 16
                             )

# summarize results
print("Best: %f using %s" % (model.best_score_, model.best_params_))
means = model.cv_results_['mean_test_score']
stds = model.cv_results_['std_test_score']
params = model.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 9 folds for each of 100 candidates, totalling 900 fits


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.


KeyboardInterrupt: 

In [43]:
def model(input_shape):
    X_inputs = keras.layers.Input(input_shape)
    X = X_inputs
    
    X = keras.layers.Conv1D(filters=16, kernel_size=64, dilation_rate=8,
                            padding='same', data_format="channels_last")(X)
    X = keras.layers.Dropout(0.5)(X)    
    X = keras.layers.AveragePooling1D(pool_size=32)(X)
    X = keras.layers.BatchNormalization(axis = 2)(X)
    X = keras.layers.Activation('relu')(X)
    
    X = keras.layers.Conv1D(filters=64, kernel_size=16, dilation_rate=2, 
                            padding='valid', data_format="channels_last")(X)
    X = keras.layers.Dropout(0.5)(X)    
    X = keras.layers.AveragePooling1D(pool_size=4)(X)
    X = keras.layers.BatchNormalization(axis = 2)(X)
    X = keras.layers.Activation('relu')(X)

    X = keras.layers.Flatten()(X)
#     X = keras.layers.Dropout(0.5)(X)    
    X = keras.layers.Dense(5, activation='softmax')(X)
    
    model = keras.models.Model(inputs=X_inputs, outputs=X, name='cnn')
    return model

In [44]:
keras.layers.Input(X_train.shape[0:])

<tf.Tensor 'input_15:0' shape=(None, 779143, 1491) dtype=float32>

In [45]:
m = model(X_train.shape[0:])

In [46]:
opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
m.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [47]:
m.summary()

Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 779143, 1491)]    0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 779143, 16)        1526800   
_________________________________________________________________
dropout_4 (Dropout)          (None, 779143, 16)        0         
_________________________________________________________________
average_pooling1d_4 (Average (None, 24348, 16)         0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 24348, 16)         64        
_________________________________________________________________
activation_4 (Activation)    (None, 24348, 16)         0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 24318, 64)         16448   

In [48]:
m.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50


ValueError: in user code:

    /opt/tljh/user/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /opt/tljh/user/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/tljh/user/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/tljh/user/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/tljh/user/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:531 train_step  **
        y_pred = self(x, training=True)
    /opt/tljh/user/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:927 __call__
        outputs = call_fn(cast_i